In [1]:
import duckdb
import pandas as pd
from deltalake import DeltaTable
import boto3
import subprocess
import sys

# ============================================================
# Cấu hình kết nối MinIO (qua DuckDB)
# ============================================================
print("🔗 Đang cấu hình kết nối tới MinIO...")

duckdb.sql("""
INSTALL httpfs;
LOAD httpfs;

SET s3_region='us-east-1';
SET s3_url_style='path';
SET s3_endpoint='127.0.0.1:9000';
SET s3_use_ssl=false;
SET s3_access_key_id='minioadmin';
SET s3_secret_access_key='minioadmin';
""")

print("✅ Cấu hình MinIO thành công.\n")

# ============================================================
# Đường dẫn Delta Lake
# ============================================================
bucket = "real-estate"
prefix = "lake/bronze/property/"
endpoint_url = "http://127.0.0.1:9000"

# ============================================================
# Liệt kê danh sách file .parquet trong MinIO (không dùng ký tự `*`)
# ============================================================
print("📁 Đang liệt kê các file Parquet thực tế...")

s3 = boto3.client(
    "s3",
    endpoint_url=endpoint_url,
    aws_access_key_id="minioadmin",
    aws_secret_access_key="minioadmin",
    region_name="us-east-1",
)

objects = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
parquet_files = [
    f"{endpoint_url}/{bucket}/{obj['Key']}"
    for obj in objects.get("Contents", [])
    if obj["Key"].endswith(".parquet")
]

if not parquet_files:
    print("⚠️ Không tìm thấy file Parquet nào trong MinIO.")
    exit()

print(f"✅ Tìm thấy {len(parquet_files)} file Parquet:")
for f in parquet_files:
    print("-", f)
print()

🔗 Đang cấu hình kết nối tới MinIO...
✅ Cấu hình MinIO thành công.

📁 Đang liệt kê các file Parquet thực tế...
✅ Tìm thấy 1 file Parquet:
- http://127.0.0.1:9000/real-estate/lake/bronze/property/part-00001-0bd52ddc-86c9-4e39-a7e0-20223d661d80-c000.snappy.parquet



In [2]:
# ============================================================
# Đọc metadata Delta Lake
# ============================================================
print("📁 Đang đọc metadata Delta Lake...")

dt = DeltaTable(
    "s3a://real-estate/lake/bronze/property",
    storage_options={
        "AWS_ACCESS_KEY_ID": "minioadmin",
        "AWS_SECRET_ACCESS_KEY": "minioadmin",
        "AWS_ENDPOINT_URL": endpoint_url,
        "AWS_ALLOW_HTTP": "true"
    }
)

print(f"📦 Version hiện tại của Delta Lake: {dt.version()}")
print("📄 Các file parquet hiện có:")
print(dt.files())
print()



📁 Đang đọc metadata Delta Lake...
📦 Version hiện tại của Delta Lake: 0
📄 Các file parquet hiện có:
['part-00001-0bd52ddc-86c9-4e39-a7e0-20223d661d80-c000.snappy.parquet']



In [3]:
# ============================================================
# Truy vấn DuckDB với danh sách file cụ thể
# ============================================================
print("\n🔍 Đang truy vấn DuckDB với danh sách file cụ thể...")

try:
    # Chúng ta dùng '?' để truyền biến Python (params) vào SQL
    # Cần dấu ngoặc kép "..." quanh tên cột vì nó có chữ hoa/thường
    result_df = duckdb.sql(
        'SELECT "propertyDetails_propertyId" FROM read_parquet(?)',
        params=[parquet_files]  
    ).df()

    print("✅ Truy vấn thành công!")
    print(f"Tìm thấy {len(result_df)} dòng.")
    print("\n--- 5 dòng đầu tiên ---")
    print(result_df.head())

except Exception as e:
    print(f"❌ Lỗi khi truy vấn DuckDB: {e}")

# ============================================================
# Truy vấn SQL
# ============================================================
property_id = result_df['propertyDetails_propertyId'].sample(n=1).iloc[0]
query = f"""
SELECT propertyDetails_propertyId, url, latitude, longitude
FROM read_parquet({parquet_files})
WHERE propertyDetails_propertyId = '{property_id}'
"""
filtered = duckdb.sql(query).df()
print(f"🔍 Dữ liệu của propertyDetails_propertyId = {property_id}:")
print(filtered)
print()


🔍 Đang truy vấn DuckDB với danh sách file cụ thể...
✅ Truy vấn thành công!
Tìm thấy 20 dòng.

--- 5 dòng đầu tiên ---
  propertyDetails_propertyId
0           a42e495ae164e98f
1           a42e495ae164e98f
2           9f525b3d48e33a58
3           9f525b3d48e33a58
4           f05138f3edbed10a
🔍 Dữ liệu của propertyDetails_propertyId = 7235a57df32a9d77:
  propertyDetails_propertyId  \
0           7235a57df32a9d77   
1           7235a57df32a9d77   

                                                 url  latitude  longitude  
0  https://nhadat247.com.vn/ban-nha-hong-mai-hai-...  21.00099  105.85053  
1  https://nhadat247.com.vn/ban-nha-hong-mai-hai-...  21.00099  105.85053  



In [6]:
# ============================================================
# Truy vấn SQL tương thích với schema mới (8 cột)
# ============================================================
property_id = result_df['propertyDetails_propertyId'].sample(n=1).iloc[0]
query = f"""
SELECT propertyDetails_propertyId, "Mức giá", "Diện tích", "Địa chỉ", url, latitude, longitude
FROM read_parquet({parquet_files})
WHERE propertyDetails_propertyId = '{property_id}'
"""
filtered = duckdb.sql(query).df()
print(f"🔍 Dữ liệu của propertyDetails_propertyId = {property_id}:")
print(filtered)
print()

🔍 Dữ liệu của propertyDetails_propertyId = 7c203910fcace6f6:
  propertyDetails_propertyId Mức giá Diện tích  \
0           7c203910fcace6f6    9 tỷ      33m2   
1           7c203910fcace6f6    9 tỷ      33m2   

                               Địa chỉ  \
0  Phường Khương Thượng,Đống Đa,Hà Nội   
1  Phường Khương Thượng,Đống Đa,Hà Nội   

                                                 url  latitude  longitude  
0  https://nhadat247.com.vn/nha-truong-chinh-chun...  21.02161  105.82323  
1  https://nhadat247.com.vn/nha-truong-chinh-chun...  21.02161  105.82323  



In [11]:
# ============================================================
# Truy vấn SQL tương thích với schema mới (8 cột)
# ============================================================
import pandas as pd

# Hiển thị đầy đủ nội dung các cột
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)

property_id = result_df['propertyDetails_propertyId'].sample(n=1).iloc[0]

query = f"""
SELECT propertyDetails_propertyId,
       "Mức giá",
       "Diện tích",
       "Địa chỉ",
       "Tiêu đề",
       url,
       latitude,
       longitude
FROM read_parquet({parquet_files})
WHERE propertyDetails_propertyId = '{property_id}'
"""

filtered = duckdb.sql(query).df()

print(f"🔍 Dữ liệu chi tiết của propertyDetails_propertyId = {property_id}:")
print("=" * 120)
print(filtered.to_string(index=False))   # In toàn bộ nội dung từng cột không bị rút gọn
print("=" * 120)
print(f"✅ Tổng số dòng: {len(filtered)}")
print()


🔍 Dữ liệu chi tiết của propertyDetails_propertyId = 0e55b3a359001128:
propertyDetails_propertyId Mức giá Diện tích                                  Địa chỉ                                                             Tiêu đề                                                                                                   url  latitude  longitude
          0e55b3a359001128  6.9 tỷ      21m2 Phố Đặng Văn Ngữ,Phường Kim Liên,Đống Đa Bán lô góc phố Đặng Văn Ngữ Đống Đa - 21m2 4 tầng ,Mt 3.4m - 6.9 tỷ https://nhadat247.com.vn/ban-lo-goc-pho-dang-van-ngu-dong-da-21m2-4-tang-mt-3-4m-6-9-ty-pid99018.html  21.01305  105.82887
          0e55b3a359001128  6.9 tỷ      21m2 Phố Đặng Văn Ngữ,Phường Kim Liên,Đống Đa Bán lô góc phố Đặng Văn Ngữ Đống Đa - 21m2 4 tầng ,Mt 3.4m - 6.9 tỷ https://nhadat247.com.vn/ban-lo-goc-pho-dang-van-ngu-dong-da-21m2-4-tang-mt-3-4m-6-9-ty-pid99018.html  21.01305  105.82887
✅ Tổng số dòng: 2



In [7]:
# ============================================================
# DESCRIBE HISTORY (Lịch sử Delta)
# ============================================================
print("🕓 DESCRIBE HISTORY (Lịch sử thay đổi bảng):")
try:
    history = dt.history()
    if history:
        for h in history:
            print(f"Version {h['version']}: {h['timestamp']} - {h['operation']}")
    else:
        print("⚠️ Không có thông tin lịch sử (Delta log trống).")
except Exception as e:
    print(f"⚠️ Lỗi khi đọc lịch sử: {e}")
print()


🕓 DESCRIBE HISTORY (Lịch sử thay đổi bảng):
Version 0: 1762589589741 - WRITE



In [8]:
# ============================================================
# Đọc dữ liệu qua DuckDB
# ============================================================
print("📦 Đang đọc dữ liệu từ MinIO (DuckDB)...")

try:
    df = duckdb.sql(f"SELECT * FROM read_parquet({parquet_files})").df()
    print(f"✅ Đọc thành công {len(df)} dòng dữ liệu.")
    print("📋 Các cột có sẵn:")
    print(df.columns.tolist())

    # --- Thống kê mô tả ---
    print("\n📊 Thống kê mô tả:")
    print(df.describe(include='all').transpose())
except Exception as e:
    print(f"❌ Lỗi khi đọc Parquet từ MinIO: {e}")
    df = pd.DataFrame()
print()

📦 Đang đọc dữ liệu từ MinIO (DuckDB)...
✅ Đọc thành công 20 dòng dữ liệu.
📋 Các cột có sẵn:
['url', 'Diện tích', 'Mức giá', 'Địa chỉ', 'Ngày đăng', 'Tiêu đề', 'latitude', 'longitude', 'propertyDetails_propertyId']

📊 Thống kê mô tả:
                           count unique  \
url                           20     10   
Diện tích                     20      9   
Mức giá                       20      8   
Địa chỉ                       20      8   
Ngày đăng                     20      1   
Tiêu đề                       20     10   
latitude                    20.0    NaN   
longitude                   20.0    NaN   
propertyDetails_propertyId    20     10   

                                                                          top  \
url                         https://nhadat247.com.vn/ban-nha-pho-khuong-di...   
Diện tích                                                                33m2   
Mức giá                                                                  5 tỷ   
Địa chỉ     

In [9]:
# ============================================================
# SELECT COUNT(*)
# ============================================================
if not df.empty:
    print("📈 SELECT COUNT(*) FROM delta:")
    count_result = duckdb.sql(f"SELECT COUNT(*) AS total_rows FROM read_parquet({parquet_files})").df()
    print(count_result)
    print()

    # ============================================================
    # DESCRIBE bảng
    # ============================================================
    print("📋 DESCRIBE (Cấu trúc cơ bản):")
    duckdb.sql(f"DESCRIBE SELECT * FROM read_parquet({parquet_files})").show()
    print()

    # ============================================================
    # 💾 Ghi ra CSV (tùy chọn)
    # ============================================================
    # df.to_csv("property_data_duckdb.csv", index=False, encoding="utf-8")
    # print("💾 Đã lưu dữ liệu ra file: property_data_duckdb.csv")

else:
    print("⚠️ Không có dữ liệu để xử lý.")

📈 SELECT COUNT(*) FROM delta:
   total_rows
0          20

📋 DESCRIBE (Cấu trúc cơ bản):
┌────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│        column_name         │ column_type │  null   │   key   │ default │  extra  │
│          varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ url                        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Diện tích                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Mức giá                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Địa chỉ                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Ngày đăng                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Tiêu đề                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ latitude                   │ DOUBLE      │ YES     │ NULL  

In [10]:
# from nbconvert import WebPDFExporter
# from traitlets.config import Config
# import nbformat

# notebook_path = "E:/Real-Estate_Project_Data_Engineering/practical-data-engineering/src/pipelines/real-estate/realestate/notebooks/TEST.ipynb"

# with open(notebook_path, "r", encoding="utf-8") as f:
#     nb_node = nbformat.read(f, as_version=4)

# c = Config()
# c.WebPDFExporter.allow_chromium_download = True

# exporter = WebPDFExporter(config=c)

# body, _ = exporter.from_notebook_node(nb_node)

# with open("TEST.pdf", "wb") as f:
#     f.write(body)

# print("✅ Đã xuất thành công file TEST.pdf")

✅ Đã xuất thành công file TEST.pdf


In [1]:
# import duckdb
# import pandas as pd
# from deltalake import DeltaTable
# import boto3
# import json

# # ============================================================
# # 0️⃣ Cấu hình hiển thị Pandas (hiện toàn bộ cột và dòng)
# # ============================================================
# pd.set_option("display.max_columns", None)      # Hiện tất cả các cột
# pd.set_option("display.max_rows", None)         # Hiện tất cả các dòng (cẩn thận nếu bảng lớn)
# pd.set_option("display.width", 0)               # Không giới hạn độ rộng in ra
# pd.set_option("display.colheader_justify", "left")

# # ============================================================
# # 1️⃣ Cấu hình kết nối MinIO (qua DuckDB)
# # ============================================================
# print("🔗 Đang cấu hình kết nối tới MinIO...")

# duckdb.sql("""
# INSTALL httpfs;
# LOAD httpfs;

# SET s3_region='us-east-1';
# SET s3_url_style='path';
# SET s3_endpoint='127.0.0.1:9000';
# SET s3_use_ssl=false;
# SET s3_access_key_id='minioadmin';
# SET s3_secret_access_key='minioadmin';
# """)

# print("✅ Cấu hình MinIO thành công.\n")

# # ============================================================
# # 2️⃣ Đường dẫn Delta Lake
# # ============================================================
# bucket = "real-estate"
# prefix = "lake/bronze/property/"
# endpoint_url = "http://127.0.0.1:9000"

# # ============================================================
# # 3️⃣ Liệt kê danh sách file .parquet trong MinIO
# # ============================================================
# print("📁 Đang liệt kê các file Parquet thực tế...")

# s3 = boto3.client(
#     "s3",
#     endpoint_url=endpoint_url,
#     aws_access_key_id="minioadmin",
#     aws_secret_access_key="minioadmin",
#     region_name="us-east-1",
# )

# objects = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
# parquet_files = [
#     f"{endpoint_url}/{bucket}/{obj['Key']}"
#     for obj in objects.get("Contents", [])
#     if obj["Key"].endswith(".parquet")
# ]

# if not parquet_files:
#     print("⚠️ Không tìm thấy file Parquet nào trong MinIO.")
#     exit()

# print(f"✅ Tìm thấy {len(parquet_files)} file Parquet:")
# for f in parquet_files:
#     print("-", f)
# print()

# # ============================================================
# # 4️⃣ Đọc metadata Delta Lake
# # ============================================================
# print("📁 Đang đọc metadata Delta Lake...")

# dt = DeltaTable(
#     "s3a://real-estate/lake/bronze/property",
#     storage_options={
#         "AWS_ACCESS_KEY_ID": "minioadmin",
#         "AWS_SECRET_ACCESS_KEY": "minioadmin",
#         "AWS_ENDPOINT_URL": endpoint_url,
#         "AWS_ALLOW_HTTP": "true"
#     }
# )

# print(f"📦 Version hiện tại của Delta Lake: {dt.version()}")
# print("📄 Các file parquet hiện có:")
# print(dt.files())
# print()

# # ============================================================
# # 5️⃣ Schema chi tiết
# # ============================================================
# print("🧱 DESCRIBE FORMATTED (Schema của bảng):")
# try:
#     schema = dt.to_pyarrow_dataset().schema
#     for field in schema:
#         print(f"- {field.name}: {field.type}")
# except Exception as e:
#     print(f"⚠️ Không thể đọc schema chi tiết: {e}")
# print()

# # ============================================================
# # 6️⃣ Lịch sử Delta
# # ============================================================
# print("🕓 DESCRIBE HISTORY:")
# try:
#     history = dt.history()
#     if history:
#         for h in history:
#             print(f"Version {h['version']}: {h['timestamp']} - {h['operation']}")
#     else:
#         print("⚠️ Không có thông tin lịch sử.")
# except Exception as e:
#     print(f"⚠️ Lỗi khi đọc lịch sử: {e}")
# print()

# # ============================================================
# # 7️⃣ Đọc toàn bộ dữ liệu (hiển thị tất cả cột)
# # ============================================================
# print("📦 Đang đọc dữ liệu từ MinIO (DuckDB)...")

# try:
#     df = duckdb.sql(f"SELECT * FROM read_parquet({parquet_files})").df()
#     print(f"✅ Đọc thành công {len(df)} dòng dữ liệu.")
#     print(f"📋 Tổng số cột: {len(df.columns)}")
#     print("\n📄 Danh sách cột:")
#     print(df.columns.tolist())
#     print("\n📊 Dữ liệu thu thập được:")
#     print(df)   # <-- Hiển thị toàn bộ bảng
# except Exception as e:
#     print(f"❌ Lỗi khi đọc Parquet từ MinIO: {e}")
#     df = pd.DataFrame()
# print()

# # ============================================================
# # 8️⃣ SELECT COUNT(*)
# # ============================================================
# if not df.empty:
#     print("📈 SELECT COUNT(*) FROM delta:")
#     count_result = duckdb.sql(f"SELECT COUNT(*) AS total_rows FROM read_parquet({parquet_files})").df()
#     print(count_result)
#     print()

#     # ============================================================
#     # 9️⃣ DESCRIBE (Cấu trúc cơ bản)
#     # ============================================================
#     print("📋 DESCRIBE (Cấu trúc cơ bản):")
#     duckdb.sql(f"DESCRIBE SELECT * FROM read_parquet({parquet_files})").show()
#     print()

# else:
#     print("⚠️ Không có dữ liệu để xử lý.")
